In [77]:
#Module 4 Challenge
#by: Nathaniel Mizzell
#07.09.2023

# Findings:
This report contains data from 15 schools, with a student population of about 39 thousand.

Overall, the average math score is 78.99%, and the average reading score is 81.88%. 

The school with the highest average math score is Pena High School, at 83.83%. Pena High School also has the highest average reading score, at 84.04%.

Accross all schools, 11th grade performs the best on math, at 79.08% (avg score). The highest performing grade for reading subject is the 9th grade, at 81.91%.

In general, students perform better on reading than math.

Suprisingly, schools that have a lower budget per student have better results than schools with a higher budget per student. Schools that receive less than $585 per studet have the higest overall passing rate, at 90.36%. The overall passing rate decreases as the budget per student increases.


In [78]:
#Import dependencies
import pandas as pd
from pathlib import Path
import warnings

#ignore warnings
warnings.filterwarnings("ignore")

In [79]:
#Get data
school_df_path = Path("Resources\schools_complete.csv")
student_df_path = Path("Resources\students_complete.csv")

school_df = pd.read_csv(school_df_path)
student_df = pd.read_csv(student_df_path)

#Merge dataframes for analysis
df = pd.merge(student_df, school_df, on='school_name', how='left')

df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [80]:
#District Summary

#Total number of unique schools
unique_school_count = school_df['school_name'].nunique()

#Total Students
total_students = school_df['size'].sum()

#Total Budget
total_budget = school_df['budget'].sum()

#Average math score
avg_math_score = df['math_score'].mean()

#Average reading score
avg_reading_score = df['reading_score'].mean()

#PCT passing math (>= 70)
num_passing_students_math = df['math_score'].loc[df['math_score'] >= 70].count()
pct_passing_math = num_passing_students_math / float(total_students)

#PCT passing reading
num_passing_students_reading = df['reading_score'].loc[df['reading_score'] >= 70].count()
pct_passing_reading = num_passing_students_reading / total_students

#PCT overall passing
num_passing_students_overall = df['reading_score'].loc[(df['reading_score'] >= 70)
                                                        & (df['math_score'] >= 70)].count()
pct_passing_overall = num_passing_students_overall / total_students


#return results in a pandas series
district_summary = pd.Series({
    'Total number of unique schools': unique_school_count
    ,'Total Students': total_students
    ,'Total Budget': total_budget
    ,'Average math score': avg_math_score
    ,'Average reading score': avg_reading_score
    ,'PCT passing math (>= 70)': pct_passing_math * 100
    ,'PCT passing reading': pct_passing_reading * 100
    ,'PCT overall passing': pct_passing_overall * 100
}).map("{:,.2f}".format)

district_summary

Total number of unique schools            15.00
Total Students                        39,170.00
Total Budget                      24,649,428.00
Average math score                        78.99
Average reading score                     81.88
PCT passing math (>= 70)                  74.98
PCT passing reading                       85.81
PCT overall passing                       65.17
dtype: object

In [81]:
#School summary

#create results DF
per_school_summary = pd.DataFrame()

#add a column to the ungrouped df. this will allow a sum aggregation to be used...
#to find the total passing students.
#set default value to 0. since we take the sum later, only passing students get 1
df['Passed Reading'] = 0
df['Passed Math'] = 0
df['Passed Overall'] = 0

#set the value of passed reading to 1 if the student passed
df['Passed Reading'].loc[df['reading_score'] >= 70] = 1
df['Passed Math'].loc[df['math_score'] >= 70] = 1
df['Passed Overall'].loc[(df['reading_score'] >= 70) & (df['math_score'] >= 70)] = 1

#Strategy: group data by school 
grouped_df = df.groupby(by='school_name')

#School name
per_school_summary['School Name'] = grouped_df['school_name'].first()

#School type
per_school_summary['School Type'] = grouped_df['type'].first()

#Total students
per_school_summary['Total Students'] = grouped_df['student_name'].count()

#Total school budget
per_school_summary['Total Budget'] = grouped_df['budget'].mean()

#Per student budget
per_school_summary['Budget per Student'] = per_school_summary['Total Budget'] / per_school_summary['Total Students']

#Average math score
per_school_summary['Avg Math Score'] = grouped_df['math_score'].mean()

#Average reading score
per_school_summary['Avg Reading Score'] = grouped_df['reading_score'].mean()

#PCT passing math
per_school_summary['Total Passing Math'] = grouped_df['Passed Math'].sum()
per_school_summary['PCT Passing Math'] = per_school_summary['Total Passing Math'] / per_school_summary['Total Students']

#PCT passing reading
per_school_summary['Total Passing Reading'] = grouped_df['Passed Reading'].sum()
per_school_summary['PCT Passing Reading'] = per_school_summary['Total Passing Reading'] / per_school_summary['Total Students']

#PCT overall passing
per_school_summary['Total Passing Overall'] = grouped_df['Passed Overall'].sum()
per_school_summary['PCT Passing Overall'] = per_school_summary['Total Passing Overall'] / per_school_summary['Total Students']

per_school_summary = per_school_summary.drop(['Total Passing Overall', 'Total Passing Reading', 'Total Passing Math'], axis=1)

per_school_summary.sort_values(by='Avg Reading Score', ascending=False).head()

,School Name,School Type,Total Students,Total Budget,Budget per Student,Avg Math Score,Avg Reading Score,PCT Passing Math,PCT Passing Reading,PCT Passing Overall
school_name,,,,,,,,,,
Pena High School,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,0.945946,0.959459,0.905405
Wilson High School,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,0.938677,0.965396,0.905826
Cabrera High School,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Wright High School,Wright High School,Charter,1800,1049400,583.0,83.682222,83.955000,0.933333,0.966111,0.903333
Thomas High School,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,0.932722,0.973089,0.909480


In [82]:
#Highest performing Schools by PCT overall passing
top_schools = per_school_summary.sort_values(by='PCT Passing Overall', ascending=False).iloc[0:5,:]
top_schools

,School Name,School Type,Total Students,Total Budget,Budget per Student,Avg Math Score,Avg Reading Score,PCT Passing Math,PCT Passing Reading,PCT Passing Overall
school_name,,,,,,,,,,
Cabrera High School,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Thomas High School,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,0.932722,0.973089,0.909480
Griffin High School,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.905995
Wilson High School,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,0.938677,0.965396,0.905826
Pena High School,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,0.945946,0.959459,0.905405


In [83]:
#Lowest performing schools by PCT overall passing
bottom_schools = per_school_summary.sort_values(by='PCT Passing Overall', ascending=True).iloc[0:5,:]
bottom_schools

,School Name,School Type,Total Students,Total Budget,Budget per Student,Avg Math Score,Avg Reading Score,PCT Passing Math,PCT Passing Reading,PCT Passing Overall
school_name,,,,,,,,,,
Rodriguez High School,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,0.663666,0.802201,0.529882
Figueroa High School,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Huang High School,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Hernandez High School,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Johnson High School,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.535392


In [84]:
#Average math Score by Grade
grouped_df = df.groupby(by='grade')

avg_math_score_by_grade = pd.Series(grouped_df['math_score'].mean())
avg_math_score_by_grade

grade
10th    78.941483
11th    79.083548
12th    78.993164
9th     78.935659
Name: math_score, dtype: float64

In [85]:
#Reading scores by grade
grouped_df = df.groupby(by='grade')

avg_reading_score_by_grade = pd.Series(grouped_df['reading_score'].mean())
avg_reading_score_by_grade

grade
10th    81.874410
11th    81.885714
12th    81.819851
9th     81.914358
Name: reading_score, dtype: float64

In [86]:
#Scores by school spending
#define buckets
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

#cut data into buckets
per_school_summary['Budget Bucket'] = pd.cut(per_school_summary['Budget per Student'], bins=spending_bins, labels=labels, include_lowest=True)

#group the dataframe
grouped_results = per_school_summary.groupby(by='Budget Bucket')

#create a new dataframe with aggregations applied to the grouped dataframe
spending_summary = pd.DataFrame()
spending_summary['Math Scores'] = grouped_results['Avg Math Score'].mean()
spending_summary['Reading Scores'] = grouped_results['Avg Reading Score'].mean()
spending_summary['PCT Passing Math'] = grouped_results['PCT Passing Math'].mean()
spending_summary['PCT Passing Reading'] = grouped_results['PCT Passing Reading'].mean()
spending_summary['PCT Passing Overall'] = grouped_results['PCT Passing Overall'].mean()

spending_summary

,Math Scores,Reading Scores,PCT Passing Math,PCT Passing Reading,PCT Passing Overall
Budget Bucket,,,,,
<$585,83.455399,83.933814,0.934601,0.966109,0.903695
$585-630,81.899826,83.155286,0.871335,0.927182,0.814186
$630-645,78.518855,81.624473,0.734842,0.843918,0.628577
$645-680,76.997210,81.027843,0.661648,0.811340,0.535269


In [87]:
#Scores by school size
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

per_school_summary['Size Bucket'] = pd.cut(per_school_summary['Total Students'], bins=size_bins, labels=labels)

grouped_results = per_school_summary.groupby(by='Size Bucket')

size_summary = pd.DataFrame()
size_summary['Math Scores'] = grouped_results['Avg Math Score'].mean()
size_summary['Reading Scores'] = grouped_results['Avg Reading Score'].mean()
size_summary['PCT Passing Math'] = grouped_results['PCT Passing Math'].mean()
size_summary['PCT Passing Reading'] = grouped_results['PCT Passing Reading'].mean()
size_summary['PCT Passing Overall'] = grouped_results['PCT Passing Overall'].mean()

size_summary

,Math Scores,Reading Scores,PCT Passing Math,PCT Passing Reading,PCT Passing Overall
Size Bucket,,,,,
Small (<1000),83.821598,83.929843,0.935502,0.960994,0.898839
Medium (1000-2000),83.374684,83.864438,0.935997,0.967907,0.906215
Large (2000-5000),77.746417,81.344493,0.699634,0.827666,0.582860


In [88]:
#Scores by school type
grouped_results = per_school_summary.groupby(by='School Type')

type_summary = pd.DataFrame()
type_summary['Math Scores'] = grouped_results['Avg Math Score'].mean()
type_summary['Reading Scores'] = grouped_results['Avg Reading Score'].mean()
type_summary['PCT Passing Math'] = grouped_results['PCT Passing Math'].mean()
type_summary['PCT Passing Reading'] = grouped_results['PCT Passing Reading'].mean()
type_summary['PCT Passing Overall'] = grouped_results['PCT Passing Overall'].mean()

type_summary

,Math Scores,Reading Scores,PCT Passing Math,PCT Passing Reading,PCT Passing Overall
School Type,,,,,
Charter,83.473852,83.896421,0.936208,0.965865,0.904322
District,76.956733,80.966636,0.665485,0.807991,0.536722
